In [1]:
# импорт библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np


In [2]:

def create_dataloader(x, y, batch_size=64, shuffle=True):
    dataset = TensorDataset(x, y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [3]:

# Упрощенный метод генерации данных
tensor_10 = torch.randint(10, (1000, 10))  # 1000 последовательностей по 10 элементов каждая


In [4]:

# Обновленная функция трансформации
def transform(tensor):
    y = torch.zeros_like(tensor)
    for j in range(len(y)):
        for i in range(len(y[1,:])):
            y[j,0] = tensor[j,0]
            y[j,i] = (tensor[j,i] + y[j,0]) % 10
    return y


In [5]:

# Функция для создания наборов данных
def create_datasets(num_samples=1000, sequence_length=10):
    # Генерация исходных данных
    x_data = torch.randint(10, (num_samples, sequence_length))

    # Преобразование данных
    y_data = transform(x_data)

    # Создание TensorDataset
    dataset = TensorDataset(x_data, y_data)
    return dataset

# Создание обучающего и тестового наборов данных
train_dataset = create_datasets()
test_dataset = create_datasets(num_samples=200)


In [6]:

# Создание обучающих и тестовых наборов данных
train_dataset_10 = create_datasets(num_samples=1000, sequence_length=10)
test_dataset_10 = create_datasets(num_samples=200, sequence_length=10)

train_dataset_40 = create_datasets(num_samples=1000, sequence_length=40)
test_dataset_40 = create_datasets(num_samples=200, sequence_length=40)

train_dataset_80 = create_datasets(num_samples=1000, sequence_length=80)
test_dataset_80 = create_datasets(num_samples=200, sequence_length=80)


In [7]:

# Создание даталоадеров для обучающих и тестовых наборов данных
train_loader_10 = create_dataloader(train_dataset_10.tensors[0], train_dataset_10.tensors[1], batch_size=64, shuffle=True)
test_loader_10 = create_dataloader(test_dataset_10.tensors[0], test_dataset_10.tensors[1], batch_size=64, shuffle=False)

train_loader_40 = create_dataloader(train_dataset_40.tensors[0], train_dataset_40.tensors[1], batch_size=64, shuffle=True)
test_loader_40 = create_dataloader(test_dataset_40.tensors[0], test_dataset_40.tensors[1], batch_size=64, shuffle=False)

train_loader_80 = create_dataloader(train_dataset_80.tensors[0], train_dataset_80.tensors[1], batch_size=64, shuffle=True)
test_loader_80 = create_dataloader(test_dataset_80.tensors[0], test_dataset_80.tensors[1], batch_size=64, shuffle=False)


In [8]:
# Шаг 2: Определение архитектур моделей

class MyRecurrentNet(nn.Module):
    def __init__(self, rnn_class, seq_len, input_size, hidden_size, num_classes):
        super(MyRecurrentNet, self).__init__()
        self.embedding = nn.Embedding(10, input_size)
        self.rnn = rnn_class(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        # Теперь 'out' имеет размерность (batch_size, seq_len, hidden_size)
        out = self.fc(out)
        # 'out' после этого имеет размерность (batch_size, seq_len, num_classes)
        return out


In [9]:
model_MyLSTM = MyRecurrentNet(nn.LSTM, seq_len=10, input_size=10, hidden_size=64, num_classes=10)
model_MyRNN = MyRecurrentNet(nn.RNN, seq_len=10, input_size=10, hidden_size=64, num_classes=10)
model_MyGRU = MyRecurrentNet(nn.GRU, seq_len=10, input_size=10, hidden_size=64, num_classes=10)


In [10]:

# Шаг 3: Обучение моделей

# Функция для обучения модели
def train_model(model, train_loader, criterion, optimizer, num_classes, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for x, y in train_loader:
            optimizer.zero_grad()
            outputs = model(x)
            # Преобразование выходных данных модели для соответствия размерности целевых данных
            outputs = outputs.view(-1, num_classes)  # Размерность теперь [batch_size * sequence_length, num_classes]
            y = y.view(-1)  # Преобразование y к одномерному вектору
            # Вычисление потерь
            loss = criterion(outputs, y.long())  # Убедитесь, что y имеет тип LongTensor
            loss.backward()
            optimizer.step()



In [22]:
# Цикл по типам моделей
for model_type in ["RNN", "LSTM", "GRU"]:
    row = f"{model_type:<10}"
    
    # Цикл по длинам последовательностей
    for seq_len, train_loader, test_loader in zip([10, 40, 80], [train_loader_10, train_loader_40, train_loader_80], [test_loader_10, test_loader_40, test_loader_80]):
        # Создаем экземпляр модели в зависимости от типа и длины последовательности
        if model_type == "RNN":
            model = MyRecurrentNet(nn.RNN, seq_len=seq_len, input_size=10, hidden_size=64, num_classes=10)
        elif model_type == "LSTM":
            model = MyRecurrentNet(nn.LSTM, seq_len=seq_len, input_size=10, hidden_size=64, num_classes=10)
        elif model_type == "GRU":
            model = MyRecurrentNet(nn.GRU, seq_len=seq_len, input_size=10, hidden_size=64, num_classes=10)
        
        # Определяем функцию потерь и оптимизатор
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Обучаем модель
        train_model(model, train_loader, criterion, optimizer, num_classes=10)
        
        # Инициализация переменных для оценки производительности
        test_loss = 0.0
        correct = 0
        total = 0
        num_classes = 10

        # Оценка производительности на тестовых данных
        with torch.no_grad():
            for x, y in test_loader:
                outputs = model(x)
                outputs = outputs.view(-1, num_classes)  # Преобразуем выходные данные модели

                # Повторяем каждый элемент y seq_len раз
                y_expanded = y.repeat_interleave(seq_len)
                y_expanded = y_expanded[:outputs.size(0)]  # Обрезаем y_expanded, чтобы соответствовать размеру outputs

                # Вычисляем потери
                test_loss += criterion(outputs, y_expanded).item()
                _, predicted = torch.max(outputs.data, 1)
                total += y.size(0) * seq_len
                correct += (predicted == y_expanded).sum().item()

                # Расчет средней потери и точности
                test_loss /= len(test_loader)
                accuracy = 100 * correct / total



        # Расчет средней потери и точности
        test_loss /= len(test_loader)
        accuracy = 100 * correct / total
        row += f"{test_loss/len(test_loader):<15.4f}{accuracy:<15.2f}"
    print(row)


RNN       0.0490         9.45           0.0482         9.40           0.0480         9.68           
LSTM      0.0493         10.30          0.0482         9.61           0.0480         9.74           
GRU       0.0487         9.30           0.0482         9.29           0.0479         10.14          


In [25]:
# Шаг 4: Оценка производительности моделей

# Создаем заголовок для таблички с результатами
print("Модель      Длина послед.  Потери (ср.)   Точность (%)")

for model_type in ["RNN", "LSTM", "GRU"]:
    # Цикл по длинам последовательностей
    for seq_len, train_loader, test_loader in zip([10, 40, 80], [train_loader_10, train_loader_40, train_loader_80], [test_loader_10, test_loader_40, test_loader_80]):
        # Создаем экземпляр модели в зависимости от типа и длины последовательности
        if model_type == "RNN":
            model = MyRecurrentNet(nn.RNN, seq_len=seq_len, input_size=10, hidden_size=64, num_classes=10)
        elif model_type == "LSTM":
            model = MyRecurrentNet(nn.LSTM, seq_len=seq_len, input_size=10, hidden_size=64, num_classes=10)
        elif model_type == "GRU":
            model = MyRecurrentNet(nn.GRU, seq_len=seq_len, input_size=10, hidden_size=64, num_classes=10)
        
        # Определяем функцию потерь и оптимизатор
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # Обучаем модель
        train_model(model, train_loader, criterion, optimizer, num_classes=10)
        
        # Инициализация переменных для оценки производительности
        test_loss = 0.0
        correct = 0
        total = 0
        num_classes = 10

        # Оценка производительности на тестовых данных
        with torch.no_grad():
            for x, y in test_loader:
                outputs = model(x)
                outputs = outputs.view(-1, num_classes)  # Преобразуем выходные данные модели

                # Повторяем каждый элемент y seq_len раз
                y_expanded = y.repeat_interleave(seq_len)
                y_expanded = y_expanded[:outputs.size(0)]  # Обрезаем y_expanded, чтобы соответствовать размеру outputs

                # Вычисляем потери
                test_loss += criterion(outputs, y_expanded).item()
                _, predicted = torch.max(outputs.data, 1)
                total += y.size(0) * seq_len
                correct += (predicted == y_expanded).sum().item()

        # Расчет средней потери и точности
        test_loss /= len(test_loader)
        accuracy = 100 * correct / total

        # Вывод результатов для текущей модели и длины последовательности
        print(f"{model_type:<10}  {seq_len:<15}  {test_loss:<15.4f}  {accuracy:<15.2f}")


Модель      Длина послед.  Потери (ср.)   Точность (%)
RNN         10               2.3561           9.60           
RNN         40               2.3169           9.49           
RNN         80               2.3084           9.54           
LSTM        10               2.3443           10.25          
LSTM        40               2.3118           9.57           
LSTM        80               2.3076           9.04           
GRU         10               2.3421           9.35           
GRU         40               2.3158           9.43           
GRU         80               2.3069           9.66           
